In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import ensemble, metrics

In [16]:
# читаем массив данных
data = pd.read_csv('lenta_example.csv')
print(len(data))
data.head()

24467


,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
1,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина
2,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,Хоккей
3,https://lenta.ru/news/2018/12/15/newyear/,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика,Деньги
4,https://lenta.ru/news/2018/12/15/gaz/,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика,Госэкономика


**Векторизация** - процесс превращения объекта в массив чисел

До:

$$ a = \text{"мама мыла раму"}, $$
$$ b = \text{"папа мыл апельсин"} $$

После:

|   | апельсин  |  мама | мыть  |  папа |  рама  |
|---|---|---|---|---|---|
|a |  0 | 1  | 1  |  0 | 1  |
| b |  1 | 0  | 1  |  1 | 0  |

In [4]:
# делаем векторизацию - превращаем текст в набор чисел
vectorizer=TfidfVectorizer(analyzer='word')
vectorizer.fit_transform(data['text'] )

<24467x195536 sparse matrix of type '<class 'numpy.float64'>'
	with 3151943 stored elements in Compressed Sparse Row format>

## Алгоритм действий
Дано: data + answers

- Делим данные: 

$$data = train\_data + test\_data$$


- Обучаем: 

$$(train\_data + answers) \xrightarrow{\text{обучение}}  model.fit() \rightarrow \textit{обученная модель} $$


- Проверяем:

$$test\_data \xrightarrow{\text{предсказание}} model.predict()  \rightarrow \textit{ответы}$$


- Оцениваем результат:

$$Accuracy = \frac{Угаданные\ верно\ ответы}{Всего\ данных}$$

In [5]:
# формируем обучающую выборку
X_train = data.tail(10000)['text']
X_train = vectorizer.fit_transform(X_train)
y_train = data.tail(10000)['topic']

# формируем тестовую выборку
X_test = data.head(10000)['text']
X_test = vectorizer.transform(X_test)
y_test = data.head(10000)['topic']

In [6]:
# обучаем модель
model = LogisticRegression()
model.fit(X_train, y_train)
result = model.predict(X_test)

In [7]:
# посмотрим на результаты
print ('train - ', model.score(X_train, y_train))
print ('test - ', metrics.accuracy_score(y_test, result))
print (metrics.classification_report(y_test, result))
print (metrics.confusion_matrix(y_test, result))

train -  0.9784
test -  0.9297
                 precision    recall  f1-score   support

    Бывший СССР       0.95      0.89      0.92      1489
            Дом       0.93      0.85      0.89       740
       Культура       0.92      0.94      0.93      1307
Наука и техника       0.93      0.94      0.94      1387
          Спорт       0.97      0.98      0.97      2216
       Ценности       0.96      0.84      0.90       784
      Экономика       0.87      0.95      0.91      2077

    avg / total       0.93      0.93      0.93     10000

[[1324    9   11   28   11    3  103]
 [   1  631   14    4    9    3   78]
 [  16    4 1229   14   14    3   27]
 [   9    0    7 1309    4    3   55]
 [  17   11    9    3 2167    0    9]
 [   1    3   54   31   17  661   17]
 [  23   24    6   24   10   14 1976]]


In [8]:
# попробуем проверить модель вручную
model.predict(vectorizer.transform(['В Омске прошла выставка жареных гвоздей']))

array(['Культура'], dtype=object)

In [9]:
# аренда жилья превысила спрос 2018 года
# на форуме инноваций Греф презентовал новую платформу для игры на бирже
# В Омске прошла выставка жареных гвоздей
# Трамп наложил новые тарифы на сталь из китая из-за этого было открыто несколько заводов

In [10]:
model.predict(vectorizer.transform([' аренда жилья превысила спрос 2018 года']))

array(['Дом'], dtype=object)